Предполагается, что в рамках данного задания ученики сами будут изучать документацию, как минимум используя функционал `tab` и `shift+tab` от Jupyter

1. Импортируйте библиотеку pandas 
2. Считайте файл `wage.csv` в DataFrame с названием `wage`. Используйте для этого функцию pd.read_csv()

`person_id` - идентификатор человека

`gender` - пол, 0 - женский, 1 - мужской

`wage` - заработная плата в рублях

In [1]:
import pandas as pd

In [76]:
wage = pd.read_csv("wage.csv")
wage

,person_id,gender,wage
0,0,1,46793.603811
1,1,1,33481.575720
2,2,1,44523.699084
3,3,1,15995.576829
4,4,0,10282.631224
...,...,...,...
1001,995,1,66503.737185
1002,996,1,9972.956272
1003,997,0,104504.616392
1004,998,1,98927.903076


3. Поменяйте колонку `gender` так, чтобы были записи `F` (female) и `M` (male) вместо 0 и 1

In [77]:
fun = lambda x: "F" if x==0 else "M"
wage.gender = wage.gender.apply(lambda x: "F" if x==0 else "M").to_frame()
wage


,person_id,gender,wage
0,0,M,46793.603811
1,1,M,33481.575720
2,2,M,44523.699084
3,3,M,15995.576829
4,4,F,10282.631224
...,...,...,...
1001,995,M,66503.737185
1002,996,M,9972.956272
1003,997,F,104504.616392
1004,998,M,98927.903076


4. Давайте посчитаем среднюю зарплату у мужчин и женщин. Для этого используйте метод `groupby` со следующим синтаксисом:

`dataframe.groupby(Название_колонки_для_группировки)[Перечисление_колонок_для_агрегации].функция_агрегации()`

In [224]:
x = wage.groupby(wage.gender)["wage"].mean()
x

gender
F    40811.235409
M    46531.457622
Name: wage, dtype: float64

5. Теперь взглянем внимательнее на данные и обнаружим, что некоторые люди попали в выборку несколько раз. 
    1. Найдите таких людей. Подсказка: `value_counts()`
    0. Убедитесь, что записи по ним с одинаковым `wage`. Возможно, тут вам пригодится функция агрегации `nunique()`, отображающая количество разных значений
    0. Избавьтесь от повторяющихся значений. Подсказка: `drop_duplicates()`

In [115]:
counts = wage.value_counts().reset_index()
counts.columns = ['person_id', 'gender', 'wage', 'count']
counts

,person_id,gender,wage,count
0,17,M,20178.985060,2
1,16,F,35448.040878,2
2,15,F,68341.332163,2
3,14,M,13434.655493,2
4,13,M,273483.609236,2
...,...,...,...,...
995,342,M,12461.850062,1
996,343,M,6634.733174,1
997,344,M,8990.765462,1
998,345,F,66368.866737,1


In [119]:
counts.loc[counts['count'] == 2]

,person_id,gender,wage,count
0,17,M,20178.985060,2
1,16,F,35448.040878,2
2,15,F,68341.332163,2
3,14,M,13434.655493,2
4,13,M,273483.609236,2
5,12,M,23547.663281,2


In [120]:
wage.nunique()

person_id    1000
gender          2
wage         1000
dtype: int64

In [121]:
wage = wage.drop_duplicates()

In [247]:
wage = wage.loc[wage['wage'] > 0]
wage = wage.dropna(subset=['wage'])

6. Теперь посмотрим внимательнее на зарплаты
    1. Охарактеризуйте имеющиеся данные по зарплатам. Подсказка: `describe`
    1. Избавьтесь от бессмысленных значений

In [248]:
wage.describe()

,person_id,wage
count,995.000000,995.000000
mean,501.844221,44306.969585
std,287.638377,54302.194392
min,0.000000,945.648458
25%,253.500000,14683.306148
50%,502.000000,27519.361794
75%,750.500000,52267.313664
max,999.000000,755320.874132


In [249]:
wage.describe().drop(['person_id'], axis=1).drop(['min', 'std'], axis=0)

,wage
count,995.000000
mean,44306.969585
25%,14683.306148
50%,27519.361794
75%,52267.313664
max,755320.874132


7. Давайте теперь посмотрим на зарплату с учетом бонуса. Для этого нам понадобится таблица `bonus.csv`. Считайте ее в переменную `bonus`. Заметьте, что она сохранена немного в другом формате, и вам понадобится уточнить параметр `sep` - разделитель записей. Сравните текущий файл с предыдущим и попробуйте решить проблему

In [250]:
bonus = pd.read_csv("bonus.csv", sep=";")
bonus

,person_id,bonus
0,905,85059.638382
1,836,7703.346074
2,287,3120.269742
3,548,5347.987142
4,575,137257.490614
...,...,...
445,488,28102.252903
446,913,55549.347647
447,616,620397.407705
448,110,14086.067488


8. Чтобы посчитать итоговую зарплату, нам нужно по каждому человеку знать и оклад, и премию. Для этого надо будет соединить (сджойнить) таблицы по `person_id`. Используйте для этого функцию `pd.merge`. Помните, что параметр `how` должен быть `'outer'`, чтобы сохранить те записи, что есть только в одной таблице. Результат запишите в новый dataframe `df`

In [251]:
df = pd.merge(left=wage, right=bonus, how='outer')
df

,person_id,gender,wage,bonus
0,0,M,46793.603811,3.332934e+04
1,1,M,33481.575720,NaN
2,2,M,44523.699084,3.192912e+06
3,3,M,15995.576829,2.196858e+04
4,4,F,10282.631224,NaN
...,...,...,...,...
994,997,F,104504.616392,5.380978e+04
995,998,M,98927.903076,NaN
996,999,M,108107.141368,NaN
997,43,NaN,NaN,2.687784e+05


9. Наконец, давайте посчитаем итоговую зарплату
    1. Замените отсутствующие записи в колонке `bonus` нулями
    1. Уберите людей без `wage` - это те "плохие" записи, от которых мы избавлялись на предыдущих шагах
    1. Сделайте новую колонку `total`, которая будет равна 12 окладам и премии
    1. Посчитайте среднюю и медианную итоговую зарплату в разрезе по полу. Подсказка: вместо функции агрегации можно написать `.agg()` и перечислить внутри нужные агрегаты

In [1]:
import math

In [257]:
sad = lambda y: 0 if math.isnan(y) else y
df.bonus = df.bonus.apply(sad).to_frame()
df = df.dropna(subset=['wage'])
df['total'] = df.wage*12+df.bonus
df.total = df.total.round()
df

,person_id,gender,wage,bonus,total
0,0,M,46793.603811,3.332934e+04,594853.0
1,1,M,33481.575720,0.000000e+00,401779.0
2,2,M,44523.699084,3.192912e+06,3727197.0
3,3,M,15995.576829,2.196858e+04,213916.0
4,4,F,10282.631224,0.000000e+00,123392.0
...,...,...,...,...,...
992,995,M,66503.737185,3.452137e+03,801497.0
993,996,M,9972.956272,3.892790e+05,508954.0
994,997,F,104504.616392,5.380978e+04,1307865.0
995,998,M,98927.903076,0.000000e+00,1187135.0


In [258]:
df.loc[df['wage'] <= 0]

,person_id,gender,wage,bonus,total


In [259]:
df.groupby(df.gender)['total'].agg(['mean', 'median'])

,mean,median
gender,,
F,570746.140244,347623.0
M,657142.497030,437500.0


10. Сохраните `df` в файл, используя метод `to_csv()`. Не записывайте индексы

In [260]:
df.to_csv('salary.csv', index=False)